# SVM Classification with XG BOOST and K-Fold Cross Validation


## Importing the libraries, clean data and convert day time to only month


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from matplotlib import dates as mpl_dates

In [ ]:
personal_dir = '/content/drive/Shareddrives/arv_msg/colab_notebooks/'

**clean*



In [ ]:
pd.set_option('use_inf_as_na', True)
df= pd.read_csv(personal_dir+'data_nocloud.csv')
df2= pd.read_csv(personal_dir+'data_2_nocloud.csv')
df.sort_values(by=['date','label'],ascending= True, inplace= True), df2.sort_values(by=['date','label'],ascending= True, inplace= True)
df=df.reset_index().iloc[:,2:-1]
df2=df2.reset_index().iloc[:,3:]
df_all= pd.concat([df,df2],axis=1)
df_all=df_all[df_all.ndvi<=1].dropna().drop('ndre', axis=1)
df_test=pd.read_csv(personal_dir+'data_test_all.csv')
df_test=df_test.fillna(0)




In [ ]:
df_all.head(20)


,date,ndvi,ndwi,gci,savi,evi,vari,gndvi,label
0,2020-01-07,0.264461,16.014964,1.443697,0.396645,0.496856,32.756943,0.413353,1
1,2020-01-07,0.341744,17.056627,1.434540,0.512535,-4.561331,35.649495,0.469038,2
2,2020-01-07,0.286602,16.366587,1.478262,0.429847,0.636649,33.969525,0.434390,3
3,2020-01-07,0.318521,17.252888,1.458552,0.477721,0.715787,34.292733,0.418954,4
4,2020-01-12,0.259601,15.869180,1.411235,0.389357,0.453689,31.533944,0.407635,1
5,2020-01-12,0.329697,16.746986,1.400596,0.494472,0.736657,34.044670,0.450201,2
6,2020-01-12,0.280198,16.174388,1.464822,0.420244,0.523677,32.634632,0.429735,3
7,2020-01-12,0.292072,17.063158,1.420624,0.438055,0.564502,33.342060,0.408967,4
8,2020-01-17,0.238182,16.231148,1.150698,0.357231,0.569950,33.616966,0.359409,1
9,2020-01-17,0.300262,17.011378,1.042167,0.450330,2.213358,34.573121,0.370396,2


In [ ]:
df_test.head()

,Unnamed: 0,ndvi,ndwi,gci,savi,evi,vari,gndvi
0,0,0.953371,8.121936,0.052340,1.429953,-4.717043,5.245197,0.732861
1,1,0.368585,12.509348,1.326151,0.552823,1.186274,28.822949,0.393208
2,2,0.256024,16.265412,1.039270,0.383990,0.621191,35.256735,0.341341
3,3,0.344614,15.216867,1.363681,0.516862,0.920329,38.655903,0.404157
4,4,0.227775,12.575929,1.158458,0.341632,0.428513,25.122235,0.366123


#Divide the date into seperate indices

In [ ]:
#int(df.iloc[0,0][8:])

In [ ]:
#split monthly index
#for k in range(df_all.shape[0]):
#  if int(df_all.iloc[k,0][8:])>15:
#    df_all.iloc[k,0]=int(df_all.iloc[k,0][5:7])*13
#  else:
#      df_all.iloc[k,0]=int(df_all.iloc[k,0][5:7])
#df_all.rename(columns = {'date':'month_criteria'}, inplace = True)
#df_all.head()

## Splitting the dataset into the Training set and Test set

In [ ]:
X = df_all.iloc[:, 1:-1].values
y = df_all.iloc[:, -1].values
X,y

(array([[ 0.26446075, 16.01496442,  1.44369708, ...,  0.49685573,
         32.75694313,  0.41335316],
        [ 0.34174377, 17.05662689,  1.43454007, ..., -4.56133068,
         35.64949514,  0.46903794],
        [ 0.28660166, 16.36658681,  1.47826215, ...,  0.63664935,
         33.9695251 ,  0.43438965],
        ...,
        [ 0.49808936, 17.45888674,  1.62493425, ...,  0.1771794 ,
         32.69839718,  0.57413246],
        [ 0.40148978, 16.21600042,  1.71761019, ...,  0.67309327,
         33.84090036,  0.50105308],
        [ 0.57032809, 15.55711521,  2.21073697, ...,  3.63222091,
         14.96148365,  0.56919402]]),
 array([1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2,
        3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4,
        1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2,
        4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2,
        3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1,
   

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 0)

In [ ]:
# select the test sample
final_test= df_test.iloc[:, 1:].values
final_test

array([[ 0.95337084,  8.12193596,  0.05234017, ..., -4.71704295,
         5.24519745,  0.73286068],
       [ 0.36858463, 12.50934754,  1.3261506 , ...,  1.18627373,
        28.82294882,  0.39320752],
       [ 0.25602379, 16.26541194,  1.03927048, ...,  0.62119148,
        35.25673546,  0.34134147],
       ...,
       [ 0.17418087, 10.55260649,  0.74174797, ...,  0.50589098,
        21.65448884,  0.27016528],
       [ 0.14407756, 10.51069791,  0.64329183, ...,  0.45120508,
        21.04649737,  0.24305998],
       [ 0.22442988, 12.21490988,  1.09048633, ...,  0.50420243,
        25.29318309,  0.35209836]])

###IMPORT XG BOOST



In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(objective='multi:softprob')

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train, X_test

(array([[-1.10697449, -0.32697597, -1.34771181, ..., -0.31418153,
          0.3224495 , -1.61680094],
        [-0.77366989, -0.09408084, -0.33940972, ..., -0.31503276,
          0.28976362, -0.55992022],
        [-0.06997517,  0.53861339,  0.07650951, ..., -0.03286517,
          1.050035  ,  0.23449143],
        ...,
        [-0.35005706, -1.19455499, -0.60164892, ..., -0.14589084,
          0.33860057, -0.4391421 ],
        [-0.77166304,  1.16682241, -0.76534956, ..., -0.18450722,
          0.54742984, -1.00173496],
        [ 1.4546928 , -0.161204  ,  1.37541646, ..., -0.04789899,
         -0.47484324,  1.5957427 ]]),
 array([[ 0.10793065,  0.94028067,  0.24382412,  0.10797364, -0.10296369,
          0.99173299,  0.14648558],
        [-0.47685678,  1.31220245, -0.65385879, -0.47687815, -0.18395261,
          0.67541097, -0.5588931 ],
        [-0.20010755, -0.97204368, -0.3593012 , -0.20002797, -0.03797317,
         -0.76297166, -0.65055203],
        [-0.65752317, -0.7894637 , -0.58868



```
# This is formatted as code
```

## Training the Support Vector Machine (SVM) Classification model on the Training set

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

## Making the Confusion Matrix for random tree algorithm


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[3 0 2 0]
 [0 6 1 3]
 [1 0 4 0]
 [2 0 0 7]]


0.6896551724137931

In [ ]:
y_pred1 = classifier.predict(sc.transform(final_test))
y_pred1
y_pred1df = pd.DataFrame(y_pred1)
y_pred1df

,0
0,2
1,1
2,4
3,3
4,1
...,...
560,1
561,4
562,1
563,1


In [ ]:
y_pred1df.columns=['crop_type']
y_pred1df.value_counts()

crop_type
1            318
4            156
3             85
2              6
dtype: int64

In [ ]:
y_pred1df.to_csv(personal_dir+'REAL_OUTPUT.csv')

## Applying k-Fold Cross Validation



In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 55.12 %
Standard Deviation: 13.65 %


In [ ]:
df_all.to_csv('/content/drive/Shareddrives/arv_msg/colab_notebooks/data_all.csv',index=False)